In [1]:
from utils.claTestSetOrganize2 import generate_image_label_csv, check_multiple_labels
import os
import sys
import argparse
import pandas as pd

In [2]:
import os
import torch
import pandas as pd
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from datetime import datetime

# 导入您项目中的自定义模块和类
from models.UnetClassifer.unet import UnetClassifier
from models.model4compare.AlexNet import AlexNet
from models.model4compare.GoogleNet import GoogleNet
from models.model4compare.VGG import VGG
from models.model4compare.NiN import NiN
# from TDSNet.TDSNet import TDSNet

from utils.tools import create_transforms  # 确保这个函数可用
from utils.MyBlock.MyCrop import MyCrop
from utils.PILResize import PILResize

import json

In [3]:
from test_cla import BreastTestDataset, modelSelector, check_breast_test_dataset, verify_breast_test_dataset
from utils.evaluate import load_data, compute_metrics, get_classification_report_df,evaluate_predictions

In [4]:
# 定义测试配置字典
test_cfg = {
    "model": "Unet",  # 模型选择，与训练时相同
    "data": "Breast",  # 数据集选择
    "csv_path": "/Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/test.csv",  # 测试CSV文件路径
    "images_dir": "/Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A",  # 测试图片根目录
    "checkpoint_path": "/Users/huiyangzheng/Downloads/Unet_Breast_20240922_133607/model/Unet_best.pth",  # 模型检查点路径
    "transforms_cfg_path": "/Users/huiyangzheng/Downloads/Unet_Breast_20240922_133607/transforms_cfg.json",
    "output_csv": "/Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/test_predictions.csv",  # 输出预测结果CSV文件路径
    "batch_size": 16,  # 测试时的批处理大小
    "num_workers": 2,  # 数据加载器的工作进程数量
    "device": "cuda" if torch.cuda.is_available() else "cpu",  # 设备选择
    "transforms_cfg": {
        "transform_test": {
            "PILResize": {"size": (400, 400)},
            "MyCrop": {}, 
            "ToTensor": {},
            "Normalize": {
                "mean": [0.4914, 0.4822, 0.4465],
                "std": [0.2023, 0.1994, 0.2010],
            },
        },
    },
    "custom_transforms": {
        'MyCrop': MyCrop,
        'PILResize': PILResize
    },
    "num_classes": 6,  # 类别数量，根据您的实际情况设置
    "class_names": ['2类', '3类', '4A类', '4B类', '4C类', '5类'],  # 类别名称列表
}

#根据transform_cfg_path读取transforms_cfg
with open(test_cfg["transforms_cfg_path"], "r") as f:
    test_cfg["transforms_cfg"] = json.load(f)
print(test_cfg["transforms_cfg"])



{'transform_train': {'MyCrop': {}, 'Resize': {'size': [256, 256]}, 'ToTensor': {}, 'Normalize': {'mean': [0.4914, 0.4822, 0.4465], 'std': [0.2023, 0.1994, 0.201]}}, 'transform_test': {'MyCrop': {}, 'Resize': {'size': [256, 256]}, 'ToTensor': {}, 'Normalize': {'mean': [0.4914, 0.4822, 0.4465], 'std': [0.2023, 0.1994, 0.201]}}}


In [5]:
# 设置根目录路径
root_directory = '/Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A'  # 根据您的实际路径修改

# 设置输出 CSV 文件路径
output_csv_path = '/Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/test.csv'

# 调用函数生成 CSV
generate_image_label_csv(
    root_dir=root_directory,
    output_csv=output_csv_path,
    include_full_path=False,  # 如果需要包含完整路径，设为 True
    encoding='utf-8'
)

cla_test_df = pd.read_csv(output_csv_path)
cla_test_df

最大标签数：3
CSV 文件已生成：/Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/test.csv


,image_name,category,class_1,x_1,y_1,w_1,h_1,class_2,x_2,y_2,w_2,h_2,class_3,x_3,y_3,w_3,h_3
0,BUSI_0645.png,5类,6,0.515009,0.368107,0.440995,0.386831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hcjz_birads5_0024_0030.jpg,5类,6,0.523203,0.275677,0.394529,0.345650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,fou_0109.jpg,5类,6,0.464884,0.538316,0.282603,0.380641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hcjz_birads5_0031_0112.jpg,5类,6,0.478748,0.490410,0.435471,0.437307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hcjz_birads4C_0078_0012.jpg,5类,6,0.464362,0.387806,0.423532,0.345570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,hcjz_birads4C_0054_0007.jpg,4C类,5,0.524220,0.385657,0.189744,0.434021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
667,hcjz_birads3_01050_0010.jpg,4C类,5,0.540506,0.270723,0.369648,0.242170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,0526.jpg,4C类,5,0.565058,0.421222,0.326972,0.373454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,hcjz_birads4C_0055_0003.jpg,4C类,5,0.380517,0.339388,0.196006,0.191846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# 调用函数检查标签数量大于1的记录
df_filtered = check_multiple_labels(output_csv_path)

df_filtered

总共有 21 张图片拥有超过1个标签。
                      image_name category  class_1       x_1       y_1  \
85                   Image07.jpg      4A类        3  0.525669  0.410160   
200   hcjz_birads2_0050_0005.jpg       2类        1  0.503023  0.277005   
208   hcjz_birads2_0081_0004.jpg       2类        1  0.518075  0.375000   
233  hcjz_birads3_01075_0006.jpg       2类        1  0.609956  0.415591   
237   hcjz_birads2_0081_0008.jpg       2类        1  0.338035  0.275466   

          w_1       h_1  class_2       x_2       y_2       w_2       h_2  \
85   0.128603  0.127807      3.0  0.420468  0.499198  0.200520  0.118182   
200  0.143349  0.106480      1.0  0.509119  0.155093  0.104514  0.112657   
208  0.182106  0.179014      1.0  0.361313  0.225309  0.087947  0.104941   
233  0.082115  0.104770      1.0  0.534251  0.453003  0.100344  0.127898   
237  0.107612  0.115742      1.0  0.487551  0.313265  0.133480  0.129632   

     class_3  x_3  y_3  w_3  h_3  label_count  
85       NaN  NaN  NaN  NaN  N

,image_name,category,class_1,x_1,y_1,w_1,h_1,class_2,x_2,y_2,w_2,h_2,class_3,x_3,y_3,w_3,h_3,label_count
85,Image07.jpg,4A类,3,0.525669,0.410160,0.128603,0.127807,3.0,0.420468,0.499198,0.200520,0.118182,NaN,NaN,NaN,NaN,NaN,2
200,hcjz_birads2_0050_0005.jpg,2类,1,0.503023,0.277005,0.143349,0.106480,1.0,0.509119,0.155093,0.104514,0.112657,NaN,NaN,NaN,NaN,NaN,2
208,hcjz_birads2_0081_0004.jpg,2类,1,0.518075,0.375000,0.182106,0.179014,1.0,0.361313,0.225309,0.087947,0.104941,NaN,NaN,NaN,NaN,NaN,2
233,hcjz_birads3_01075_0006.jpg,2类,1,0.609956,0.415591,0.082115,0.104770,1.0,0.534251,0.453003,0.100344,0.127898,NaN,NaN,NaN,NaN,NaN,2
237,hcjz_birads2_0081_0008.jpg,2类,1,0.338035,0.275466,0.107612,0.115742,1.0,0.487551,0.313265,0.133480,0.129632,NaN,NaN,NaN,NaN,NaN,2
241,hcjz_birads2_0350_0004.jpg,2类,1,0.588952,0.222989,0.077604,0.063272,1.0,0.539802,0.258482,0.076573,0.063272,NaN,NaN,NaN,NaN,NaN,2
252,sec_0067.jpg,2类,1,0.664791,0.258985,0.093602,0.084237,1.0,0.378608,0.276165,0.182345,0.093649,NaN,NaN,NaN,NaN,NaN,2
329,fou_0001.jpg,4B类,4,0.360567,0.411082,0.234021,0.279278,4.0,0.558022,0.353402,0.145296,0.132577,NaN,NaN,NaN,NaN,NaN,2
335,fou_0005.jpg,4B类,4,0.422036,0.395515,0.204768,0.210619,4.0,0.596521,0.372835,0.134536,0.149794,NaN,NaN,NaN,NaN,NaN,2
416,0114.jpg,3类,2,0.235541,0.416896,0.130132,0.095167,2.0,0.490393,0.470120,0.087961,0.095156,NaN,NaN,NaN,NaN,NaN,2


In [7]:
# 读取CSV
df = pd.read_csv(test_cfg["csv_path"])
print(f"Total test samples: {len(df)}")

df

Total test samples: 671


,image_name,category,class_1,x_1,y_1,w_1,h_1,class_2,x_2,y_2,w_2,h_2,class_3,x_3,y_3,w_3,h_3
0,BUSI_0645.png,5类,6,0.515009,0.368107,0.440995,0.386831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hcjz_birads5_0024_0030.jpg,5类,6,0.523203,0.275677,0.394529,0.345650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,fou_0109.jpg,5类,6,0.464884,0.538316,0.282603,0.380641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hcjz_birads5_0031_0112.jpg,5类,6,0.478748,0.490410,0.435471,0.437307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hcjz_birads4C_0078_0012.jpg,5类,6,0.464362,0.387806,0.423532,0.345570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,hcjz_birads4C_0054_0007.jpg,4C类,5,0.524220,0.385657,0.189744,0.434021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
667,hcjz_birads3_01050_0010.jpg,4C类,5,0.540506,0.270723,0.369648,0.242170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,0526.jpg,4C类,5,0.565058,0.421222,0.326972,0.373454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,hcjz_birads4C_0055_0003.jpg,4C类,5,0.380517,0.339388,0.196006,0.191846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# 定义图像变换
transform_test = create_transforms(test_cfg["transforms_cfg"]["transform_test"], 
                                    custom_transforms=test_cfg["custom_transforms"])# check_breast_test_dataset(test_dataset)

print(transform_test)

Compose(
    MyCrop()
    Resize(size=[256, 256], interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.201])
)


In [9]:
# 定义图像变换
transform_test = create_transforms(test_cfg["transforms_cfg"]["transform_test"], 
                                    custom_transforms=test_cfg["custom_transforms"])

# 创建测试数据集和数据加载器
test_dataset = BreastTestDataset(csv_file=test_cfg["csv_path"], 
                                    images_dir=test_cfg["images_dir"], 
                                    transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=test_cfg["batch_size"], 
                            shuffle=False, num_workers=test_cfg["num_workers"])

verify_breast_test_dataset(test_dataset, test_cfg["images_dir"], test_cfg["csv_path"])

# 加载模型
model = modelSelector(test_cfg["model"], test_cfg["checkpoint_path"], 
                        test_cfg["num_classes"], test_cfg["device"])
print(f"Loaded model {test_cfg['model']} from {test_cfg['checkpoint_path']}")

# 初始化预测列表
predictions = []

# 进行预测
with torch.no_grad():
    for images, indices in tqdm(test_loader, desc="Predicting"):
        images = images.to(test_cfg["device"])
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        preds = preds.cpu().numpy()
        predictions.extend(preds)
        
# 将预测结果映射为类别名称
# 确保class_names的索引与模型输出对应
class_names = test_cfg["class_names"]
df['predicted_category'] = [class_names[p] for p in predictions]

# 保存到新的CSV文件
os.makedirs(os.path.dirname(test_cfg["output_csv"]), exist_ok=True)
df.to_csv(test_cfg["output_csv"], index=False)  

df

Total images found (filtered by extensions {'.png', '.gif', '.jpeg', '.bmp', '.jpg'}): 671
===== 图片路径映射示例 =====
BUSI_0645.png: /Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/5类/images/BUSI_0645.png
hcjz_birads5_0024_0030.jpg: /Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/5类/images/hcjz_birads5_0024_0030.jpg
fou_0109.jpg: /Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/5类/images/fou_0109.jpg
hcjz_birads5_0031_0112.jpg: /Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/5类/images/hcjz_birads5_0031_0112.jpg
hcjz_birads4C_0078_0012.jpg: /Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/5类/images/hcjz_birads4C_0078_0012.jpg
hcjz_birads5_0007_0024.jpg: /Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/5类/images/hcjz_bi

/Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/utils/removeFrame.py:111: UserWarning: width too small after cropped!
  warnings.warn('width too small after cropped!')
/Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/utils/removeFrame.py:114: UserWarning: height too small after cropped!
  warnings.warn('height too small after cropped!')


Image 'BUSI_0571.png' at index 100: /Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/4A类/images/BUSI_0571.png
Image shape: (425, 475)
Image 'hcjz_birads2_0050_0005.jpg' at index 200: /Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/2类/images/hcjz_birads2_0050_0005.jpg
Image shape: (1090, 858)
Image 'hcjz_birads4B_0055_0013.jpg' at index 300: /Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/4B类/images/hcjz_birads4B_0055_0013.jpg
Image shape: (1090, 858)
Image 'hcjz_birads3_01115_0003.jpg' at index 400: /Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/3类/images/hcjz_birads3_01115_0003.jpg
Image shape: (1135, 761)
Image 'thi_0260.jpg' at index 500: /Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/data/test_A/乳腺分类测试集A/A/3类/images/thi_0260.jpg
Image shape: (1552, 873)
Image '0295.jpg' at

/Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/test_cla.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(check

Loaded model Unet from /Users/huiyangzheng/Downloads/Unet_Breast_20240922_133607/model/Unet_best.pth


Predicting:   0%|          | 0/42 [00:00<?, ?it/s]/Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/utils/removeFrame.py:111: UserWarning: width too small after cropped!
  warnings.warn('width too small after cropped!')
/Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/utils/removeFrame.py:114: UserWarning: height too small after cropped!
  warnings.warn('height too small after cropped!')
/Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/utils/removeFrame.py:111: UserWarning: width too small after cropped!
  warnings.warn('width too small after cropped!')
/Users/huiyangzheng/Desktop/Project/Competition/GCAIAEC2024/AIC/TDS-Net/utils/removeFrame.py:114: UserWarning: height too small after cropped!
  warnings.warn('height too small after cropped!')
Predicting: 100%|██████████| 42/42 [02:53<00:00,  4.13s/it]


,image_name,category,class_1,x_1,y_1,w_1,h_1,class_2,x_2,y_2,w_2,h_2,class_3,x_3,y_3,w_3,h_3,predicted_category
0,BUSI_0645.png,5类,6,0.515009,0.368107,0.440995,0.386831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5类
1,hcjz_birads5_0024_0030.jpg,5类,6,0.523203,0.275677,0.394529,0.345650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4C类
2,fou_0109.jpg,5类,6,0.464884,0.538316,0.282603,0.380641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5类
3,hcjz_birads5_0031_0112.jpg,5类,6,0.478748,0.490410,0.435471,0.437307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5类
4,hcjz_birads4C_0078_0012.jpg,5类,6,0.464362,0.387806,0.423532,0.345570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4B类
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,hcjz_birads4C_0054_0007.jpg,4C类,5,0.524220,0.385657,0.189744,0.434021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4C类
667,hcjz_birads3_01050_0010.jpg,4C类,5,0.540506,0.270723,0.369648,0.242170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3类
668,0526.jpg,4C类,5,0.565058,0.421222,0.326972,0.373454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5类
669,hcjz_birads4C_0055_0003.jpg,4C类,5,0.380517,0.339388,0.196006,0.191846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4C类


In [10]:
df.to_csv(test_cfg["output_csv"], index=False)  

In [12]:
evaluate_predictions(test_cfg["output_csv"])

评估指标:
准确率 (Accuracy): 0.6557
精确率 (Precision): 0.6482
召回率 (Recall): 0.6557
F1分数 (F1-Score): 0.6450

分类报告:
              precision    recall  f1-score     support
2类             0.825000  0.792000  0.808163  125.000000
3类             0.696864  0.840336  0.761905  238.000000
4A类            0.560000  0.347107  0.428571  121.000000
4B类            0.511905  0.530864  0.521212   81.000000
4C类            0.492754  0.500000  0.496350   68.000000
5类             0.611111  0.578947  0.594595   38.000000
accuracy       0.655738  0.655738  0.655738    0.655738
macro avg      0.616272  0.598209  0.601799  671.000000
weighted avg   0.648185  0.655738  0.644971  671.000000
